In [1]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "my_database1", "my_table4s")
_ = (
    storage.schema_builder()
    ## 主键，字符串
    .add_field("_id", DataType.STRING)
    ## 可以作为where条件过滤的字段
    .add_field("name", DataType.STRING)
    ## 可以作为搜索字段
    .add_field("content", DataType.STRING, [FieldOption.ANALYZE])
    ## 不需要索引的字段，会原模原样存储
    .add_field("raw_content", DataType.STRING, [FieldOption.NO_INDEX])    
    ## 数组字段
    .add_array_field("summary", DataType.FLOAT)    
    ## 需要排序的字段
    .add_field("created_time", DataType.LONG, [FieldOption.SORT])    
    .execute()
)

## st(
# field(_id,long),
# field(name,string),
# field(content,string,analyze),
# field(raw_content,string,no_index),
# field(summary,array(float)),
# field(created_time,long,sort)
# )

# 准备数据
data = [
    {"_id": "1", "name": "Hello", "content": "Hello, world!", "raw_content": "Hello, world!", "summary": "hello world", "created_time": 1612137600},
    {"_id": "2", "name": "Byzer", "content": "Byzer, world!", "raw_content": "Byzer, world!", "summary": "byzer", "created_time": 1612137601},
    {"_id": "3", "name": "AI", "content": "AI, world!", "raw_content": "AI, world!", "summary": "AI", "created_time": 16121376002},
    {"_id": "4", "name": "ByzerAI", "content": "ByzerAI, world!", "raw_content": "ByzerAI, world!", "summary": "ByzerAi", "created_time": 16121376003},
]

# 写入Storage
storage.write_builder().add_items(data,vector_fields=["summary"],search_fields=["content"]).execute()

# commit 写入
storage.commit()

# 搜索
query = storage.query_builder()
query.set_vector_query("ByzerAi",fields=["summary"])
results = query.set_search_query("Hello",fields=["content"]).execute()
print(results)

## 过滤之后再做搜索
query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
query.set_vector_query("ByzerAi",fields="summary")
results = query.set_search_query("Hello",fields=["content"]).execute()
print(results)

## 过滤再做排序
query = storage.query_builder()
query.and_filter().add_condition("name","AI").build().sort("created_time",SortOption.DESC)
results = query.execute()
print(results)

## 根据ID做删除
storage.delete_by_ids(["3"])

query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
results = query.execute()
print("====")
print(results)

## 删除表
storage.drop_table()

2024-07-06 12:45:50.026 | INFO     | byzerllm.utils.connect_ray:connect_cluster:37 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-07-06 12:45:50,198	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-07-06 12:45:50,211	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


21


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dm/0xljd5nn10b7bwwmwv8w5v100000gn/T/jieba.cache
Loading model cost 0.668 seconds.
Prefix dict has been built successfully.
2024-07-06 12:45:51,845	WARNING __init__.py:158 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.


[{'name': 'Hello', 'created_time': 1612137600, 'raw_content': 'Hello, world!', '_id': '1', '_score': 0.032539684}, {'name': 'ByzerAI', 'created_time': 16121376003, 'raw_content': 'ByzerAI, world!', '_id': '4', '_score': 0.016666668}, {'name': 'Byzer', 'created_time': 1612137601, 'raw_content': 'Byzer, world!', '_id': '2', '_score': 0.016393442}, {'name': 'AI', 'created_time': 16121376002, 'raw_content': 'AI, world!', '_id': '3', '_score': 0.016129032}]
[{'name': 'AI', 'created_time': 16121376002, 'raw_content': 'AI, world!', '_id': '3', '_score': 0.033333335}]
[{'name': 'AI', 'created_time': 16121376002, 'raw_content': 'AI, world!', '_id': '3'}]
{'and': [{'field': 'name', 'value': 'AI'}]}
====
[{'name': 'AI', 'created_time': 16121376002, 'raw_content': 'AI, world!', '_id': '3'}]


(raylet) java.lang.ClassCastException: class java.lang.String cannot be cast to class java.lang.Long (java.lang.String and java.lang.Long are in module java.base of loader 'bootstrap')
(raylet) 	at tech.mlsql.retrieval.schema.SchemaUtils.toLuceneQuery(SchemaUtils.java:188)
(raylet) 	at io.ray.runtime.RayNativeRuntime.nativeRunTaskExecutor(Native Method)
(raylet) WARNING: Using incubator modules: jdk.incubator.vector


(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 709, in <module>
    monitor.run()
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 584, in run
    self._run()
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 438, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



In [5]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption
storage = ByzerStorage("byzerai_store", "my_database1", "my_table4s")
## 根据过滤条件删除数据
storage.query_builder().and_filter().add_condition("name","AI").build().delete()
# storage.delete_by_ids(["3"])

query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
results = query.execute()
print("====")
print(results)

{'and': [{'field': 'name', 'value': 'AI'}]}
====
[]


: 

In [1]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "memory", "memory")
_ = (
    storage.schema_builder()
    ## 主键，字符串
    .add_field("_id", DataType.STRING)
    ## 可以作为where条件过滤的字段
    .add_field("name", DataType.STRING)
    ## 可以作为搜索字段
    .add_field("content", DataType.STRING, [FieldOption.ANALYZE])
    ## 不需要索引的字段，会原模原样存储
    .add_field("raw_content", DataType.STRING, [FieldOption.NO_INDEX])    
    ## 数组字段
    .add_array_field("summary", DataType.FLOAT)    
    ## 需要排序的字段
    .add_field("created_time", DataType.LONG, [FieldOption.SORT])    
    .execute()
)



2024-07-29 14:00:23.540 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-07-29 14:00:23,688	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-07-29 14:00:23,703	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


21


In [4]:
import time

int(time.time())


1722233112

In [16]:
import time
import uuid
def memorize(name:str,s:str):
    data = [
        {"_id": str(uuid.uuid4()), "name": name, "content": s, "raw_content": s, "summary": s, "created_time": int(time.time())},    
    ]
    storage.write_builder().add_items(data,vector_fields=["summary"],search_fields=["content"]).execute()

def recall(name:str,query:str):
    searcher = storage.query_builder()
    searcher.and_filter().add_condition("name",name).build()
    searcher.set_vector_query(query,fields="summary")
    results = searcher.set_search_query(query,fields=["content"]).execute() 
    return results

In [22]:
storage.delete_by_ids(["1"])

In [18]:
memorize("william","William的生日是2月1号")

In [23]:

recall("william","William生日是几号")

[{'name': 'william',
  'created_time': 1722233443,
  'raw_content': 'William的生日是2月1号',
  '_id': 'f7fcb7b6-e5d2-493d-99c4-b1a48c23e91a',
  '_score': 0.033333335}]

In [28]:
import byzerllm
import json

llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")

@byzerllm.prompt(llm=llm)
def ask_about_william(query:str)->str:
    '''
    根据下面的信息回答用户的问题。

    {{ context }}

    用户的问题：

    {{ query }}
    '''
    return {
        "context": json.dumps(recall("william",query),ensure_ascii=False)
    }

ask_about_william("William的生日是几号")

2024-07-29 14:14:24.362 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-07-29 14:14:24,469	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-07-29 14:14:24,471	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


'根据提供的信息，William的生日是2月1号。'